In [1]:
from IPython.display import display, HTML

In [2]:
print()
js = "<script>(function($) {    function gasFlame() {var colors = ['#fff', '#99f', '#00f','#f00', '#009', '#f90'];        if (Math.random() > 0.90) {            colors.push(colors[4]);        }        var hv = 0.04,            textShadows = [],            c = 0,            h = 0,            s = 0;        while (c < colors.length - 1) {            s = 2 + Math.round(Math.random() * 2);            while (s--) {                shadow = '0 ' + h + 'em ' + -h + 'em ' + colors[c];                textShadows.push(shadow);                h -= hv;            }            c++;        }        $(this).css({            color: colors[0],            textShadow: textShadows.join(', ')        });    }      $.fn.ignite = function() {        return this.each(function() {            var letters = $(this).text().split('');            $(this).html('<span>' + letters.join('</span><span>') + '</span>');            $spans = $(this).find('span');            setInterval(function() {                $spans.each(gasFlame);            }, 100);        });    };})(jQuery);$(function() {    $('ah1').ignite();});</script><h1><ah1 style=\"font-size: 3em !important;\">"
js += "Reinforcement Learning!"
js += "</ah1>"
js += '''
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="show code..." style="font-size:10px; margin: auto;"></form>'''
display(HTML(js))
print(" Reinforcement Learning: it lit... By Alex, Gherardo, Other Alex, and Jimmy")

 Reinforcement Learning: it lit... By Alex, Gherardo, Other Alex, and Jimmy


### Project Summary
Our goal in this project was to balance an inverted pendulum.
<br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Cart-pendulum.svg/300px-Cart-pendulum.svg.png" style="width:200px; height:200px;">

This problem is most naturally suited to reinforcement learning because there is no previous dataset, but rather a reward function. It's good if the pendulum is pointed up, and it's bad if it's at an angle.

We originally tried to create a neural network, but that proved overcomplicated for this situation. This led us to attempt Q-learning. Q-Learning is a tabular method of reinforcement learning.

### Interfacing with the robot
The turtlebot uses ROS(Robot operating system). Then it operates on a publisher subscriber system. Unfortunately, this system can cause lags and loss in communication.

In [4]:
import rospy
import roslib
ser = serial.Serial('/dev/ttyACM0', 9600)
rospy.init_node('goalieBot', anonymous=True)
movement = rospy.Publisher("/mobile_base/commands/velocity", Twist, queue_size=None, tcp_nodelay=True, latch=True)
timePerStep = 8
r = rospy.Rate(timePerStep)
moveCmd = Twist()

ImportError: No module named 'rospy'

# Learner class
The Learner class is what allows the robot to use reinforcement learning. This class receives the reward and state. It then decides the action which should be taken next in order to maximize the expected reward. Then, it obser updates the q-table.
* `num_bins`: the number of discrete values we divide our angle measurements
* `num_actions`: the number of discrete values divide our possible movements
* `q-matrix`: our q-table (with the shape [`num_bins`, `num_bins`, `num_actions`])
* `epsilon`: the probability of making a random action
* `gamma`: the importance of the future rewards
* `alpha`: learning rate of the robot
* `epsilonDecay`: decreases the need of randomness as the model becomes more trained
* `state`: a 2D array with one axis representing the previous angle measures and the other representing the current angle measures. Having both measures allowed us to take into account the angular velocity of the inverted pendulum
* `actionIndex`: the index in our `q-matrix` of the action that would maximize our expected reward

In [23]:
class Learner:
    num_bins = 10
    num_actions = 30
    def __init__(self):
        num_states = 2
        self.qmatrix = np.zeros((Learner.num_bins+1, Learner.num_bins+1, Learner.num_actions+1))
        self.epsilon = .7
        self.gamma = .75
        self.alpha = .8
        self.epsilonDecay = .99
        self.state = [0, 0]
        self.actionIndex = 0

    def getMove(self, newState, reward):
        if random.random() < self.epsilon: # allows for exploration (chooses a random action)
            return random.randint(0,Learner.num_actions)
        newActionIndex = self.qmatrix[newState[0]][newState[1]].argsort()[-1]
        #alpha = 1
        self.qmatrix[int(self.state[0])][int(self.state[1])][int(self.actionIndex)] = float(self.qmatrix[self.state[0]][self.state[1]][self.actionIndex]) + \
                                    reward + self.gamma * float(self.qmatrix[int(newState[0])][int(newState[1])][int(newActionIndex)])
        self.actionIndex = newActionIndex
        self.state = newState
        self.epsilon *= self.epsilonDecay
        return newActionIndex

# Implementation

In [ ]:
speed = 0
reward = 0

# Turn a value into its nearest bin
def to_bin(value, bins):
    return np.digitize(x=[value], bins=bins)[0]

# Keep reading from serial until a real number is read.
# Necessary because Serial isn't always reliable in its reading.
def getAngle():
    global speed
    a=float('inf')
    while a == float('inf'):
        try:
            a = int(ser.readline().decode().strip())
        except:
            print("Failed reading...Trying again...")
    return a

# Execute action in the environment
def makeMove(action):
    global moveCmd
    moveCmd = Twist()
    moveCmd.linear.x = action
    movement.publish(moveCmd)

# Gets a reward - we tried two different methods.
# This method turned out to be real trouble.
def getReward():
    global reward
    
    # Method 1 - get the highest score that is obtained
    # in between each timestep. In case by the time it reads
    # the pendulum has already gone up and fallen back down.
    # The reward is reset to -1 in each step below
    reward = max(reward, 100/getAngle())
    
    # Method 2 - keep a running average of the past 10 angles.
    # Prevents faulty readings from causing catostrophic forgetting.
    rewardArr = [0 for x in range(10)]
    while True:
        del rewardArr[0]
        rewardArr.append(1000/(abs(getAngle())+1))
        reward = np.sum(rewardArr)/len(rewardArr)
        time.sleep(.1)

def cart_pole_with_qlearning():
    global speed, reward
    learner = Learner()
    angleBins = np.array(list(np.linspace(-60,70,Learner.num_bins)))
    actionBins = np.array(list(np.linspace(-1.5,1.5,Learner.num_actions+1)))
    previousAngle = getAngle()

    try:
        print("Trying to load matrix...", end='')
        learner.qmatrix = pickle.load(open("qMatrix.pkl", 'rb'))
        print("Success!")
    except:
        print("Failed.")
    # we are not going to differentiate between episode and step because,
    # when we train the robot, we let it run without ever stopping it
    # we are not going to differentiate between episode and step because,
    # when we train the robot, we let it run without ever stopping it
    # at each episode (i.e. moment when robot fails)
    #
    # In Reinforcement terminology: Our failure state is the same as our starting
    # state, so we do not need the reset stage that occurs in between each epoch
    
    thread = Thread(target = getReward) # asynchronously start reward function to allow further precision in it.
    thread.start()
    for step in range(10000):
        currentAngle = getAngle()
        state = [to_bin(currentAngle, angleBins), to_bin(previousAngle, angleBins)] # get current angle and put it in bin
        print("Reward:", reward)
        action = actionBins[learner.getMove(state, reward)] # decide on the action
        reward = -1 # necessary for method 1 of the reward function
        makeMove(action)
        previousAngle = currentAngle
        time.sleep(1/timePerStep)
        #Save the Q-Matrix in between epochs.
        if step%5 == 0:
            print("Saving the q-matrix")
            pickle.dump(learner.qmatrix, open("qMatrix.pkl", 'wb'))

if __name__ == "__main__":
    cart_pole_with_qlearning()